In [33]:
from IPython import display
import torch
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import roboflow
import ultralytics
from ultralytics import YOLO

In [34]:
display.clear_output()
ultralytics.checks()

Ultralytics 8.3.104  Python-3.8.10 torch-2.4.1+cpu CPU (AMD Ryzen 5 3600 6-Core Processor)
Setup complete  (12 CPUs, 15.9 GB RAM, 100.1/931.5 GB disk)


In [35]:
# Em apple sillicon, verificar se o MPS está disponível
print(torch.backends.mps.is_available())

False


In [36]:
# descarregar o dataset do roboflow, depois de etiquetadas as imagens e criado o dataset em Mac tive problemas de permissões e foi necessário dar permissões à pasta de conf. do roboflow:
# - sudo mkdir /Users/davidecarneiro/.config/roboflow (criar pasta onde vai guardar a conf.)
# - sudo chown -R davidecarneiro:staff ~/.config/roboflow (dar permissões ao meu user)
roboflow.login()

# criar um ficheiro com a key da api do roboflow, ou simplesmente substituir abaixo
with open('api_key', "r") as file:
    api_key = file.read().strip()

rf = roboflow.Roboflow(api_key)

You are already logged into Roboflow. To make a different login,run roboflow.login(force=True).


RuntimeError: {
    "error": {
        "message": "This API key does not exist (or has been revoked).",
        "status": 401,
        "type": "OAuthException",
        "hint": "You may retrieve your API key via the Roboflow Dashboard. Go to Account > Roboflow Keys to retrieve yours.",
        "key": "6023bd44-2251-41b1-91d4-a7c857a93e18"
    }
}

In [ ]:
# substituir nome do workspace e do projeto
project = rf.workspace("heyliceeee").project("candy-crush-saga-v3")
# se versão do dataset > 1, substituir pela versão correspondente
dataset = project.version(1).download("yolov8")
# WARN: necessário verificar os paths no ficheiro data.yaml, após este ser descarregado